In [1]:
import sys
import datetime
import time
from datetime import timedelta, date, datetime
from dateutil.relativedelta import relativedelta
import os
import urllib
import tarfile
import xarray as xr
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from math import sin, cos, sqrt, atan2, radians

import glob

from scipy import stats

import warnings
warnings.filterwarnings("ignore")

import geopandas as gpd

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

from netCDF4 import Dataset
from mpl_toolkits.basemap import Basemap
from matplotlib import cm
import haversine as hs
from shapely.geometry import MultiPoint, Point

from matplotlib.animation import FuncAnimation

In [2]:
ROx = pd.read_excel('../data_input/ROx_GCv12.xlsx', sheet_name='031924_ROx_list', header=None)
ROx = ROx.iloc[:, 0].tolist()
print(f'Total # of ROx species is {len(ROx)}.')
sorted(ROx)
ROx

Total # of ROx species is 34.


['A3O2',
 'ATO2',
 'B3O2',
 'BRO2',
 'DHPCARP',
 'DIBOO',
 'ETO2',
 'HC5OO',
 'HO2',
 'HPC52O2',
 'IEPOXOO',
 'INO2',
 'ISNOOA',
 'ISNOOB',
 'ISNOHOO',
 'ISOPNBO2',
 'ISOPNDO2',
 'KO2',
 'LIMO2',
 'MAOPO2',
 'MO2',
 'MOBAOO',
 'MRO2',
 'OH',
 'OTHRO2',
 'PIO2',
 'PO2',
 'PRN1',
 'R4N1',
 'R4O2',
 'RIO2',
 'TRO2',
 'VRO2',
 'XRO2']

In [3]:
file = open("/projectsp/f_xj103_1/jqshen/Code.12.7.1.PHCHO_Original/KPP/Tropchem/gckpp_Monitor.F90","r")
lines = file.readlines()

In [4]:
idx_list = []
line_list = []
diag = []
ROx_eq_num = []
i = 118
while i < 823:
    line = lines[i]
    line = line.replace('"', '')
    line = line.replace("'", '') 
    if '-->' in line:
        spec, _ = line.split('-->')
        select = 0
        if '+' in spec:
            spec_left, spec_right = spec.split('+')
            spec_left = spec_left.strip()
            spec_right = spec_right.strip()
            if (spec_left in ROx) & (spec_right in ROx):
                select = 1
        elif ' 2 ' in spec:
            _, speci = spec.split(' 2')
            speci = speci.strip()
            if speci in ROx:
                select = 1
        if select:    
            if '& ! index' in line:
                _, idx = line.split('& ! index')
                idx = idx.strip()
                idx_list.append(idx)
                line_list.append(i)
                ROx_eq_num.append(f"RxnRate_EQ{int(idx):03d}")
                diag.append(f"'RxnRate_EQ{int(idx):03d}                           ', 'GIGCchem',")
            else:
                i += 1
                idx = lines[i].split(':: ')[1][:-7]
                idx_list.append(idx)
                line_list.append(i-1)
                ROx_eq_num.append(f"RxnRate_EQ{idx}")
                diag.append(f"'RxnRate_EQ{idx}                           ', 'GIGCchem',")
    i += 1
print('RXN index: ', idx_list, '\n\nline #: ', line_list)

RXN index:  ['6', '7', '8', '11', '17', '18', '19', '75', '78', '79', '83', '84', '85', '101', '102', '103', '104', '105', '106', '107', '108', '109', '111', '112', '113', '115', '116', '118', '119', 'EQN_NAMES_4 ', '121', '122', '123', '124', '125', '126', '128', '129', '130', '133', '134', '135', '136', '137', '138', '139', '140', '195', '205', '206', '207', '364', '368', '370', '381', '385', '433', '435', '437', '442', '443', '454', '455', '484', '485'] 

line #:  [123, 124, 125, 128, 134, 135, 136, 194, 197, 198, 202, 203, 204, 221, 222, 223, 224, 225, 226, 227, 228, 229, 231, 232, 233, 235, 236, 238, 239, 240, 242, 243, 244, 245, 246, 247, 249, 250, 251, 254, 255, 256, 257, 258, 259, 260, 261, 318, 328, 329, 330, 493, 497, 499, 510, 514, 564, 566, 568, 573, 574, 586, 587, 617, 618]


In [5]:
ROx_eq_num = ['RxnRate_EQ120' if x == 'RxnRate_EQEQN_NAMES_4 ' else x for x in ROx_eq_num]
ROx_eq_num

['RxnRate_EQ006',
 'RxnRate_EQ007',
 'RxnRate_EQ008',
 'RxnRate_EQ011',
 'RxnRate_EQ017',
 'RxnRate_EQ018',
 'RxnRate_EQ019',
 'RxnRate_EQ075',
 'RxnRate_EQ078',
 'RxnRate_EQ079',
 'RxnRate_EQ083',
 'RxnRate_EQ084',
 'RxnRate_EQ085',
 'RxnRate_EQ101',
 'RxnRate_EQ102',
 'RxnRate_EQ103',
 'RxnRate_EQ104',
 'RxnRate_EQ105',
 'RxnRate_EQ106',
 'RxnRate_EQ107',
 'RxnRate_EQ108',
 'RxnRate_EQ109',
 'RxnRate_EQ111',
 'RxnRate_EQ112',
 'RxnRate_EQ113',
 'RxnRate_EQ115',
 'RxnRate_EQ116',
 'RxnRate_EQ118',
 'RxnRate_EQ119',
 'RxnRate_EQ120',
 'RxnRate_EQ121',
 'RxnRate_EQ122',
 'RxnRate_EQ123',
 'RxnRate_EQ124',
 'RxnRate_EQ125',
 'RxnRate_EQ126',
 'RxnRate_EQ128',
 'RxnRate_EQ129',
 'RxnRate_EQ130',
 'RxnRate_EQ133',
 'RxnRate_EQ134',
 'RxnRate_EQ135',
 'RxnRate_EQ136',
 'RxnRate_EQ137',
 'RxnRate_EQ138',
 'RxnRate_EQ139',
 'RxnRate_EQ140',
 'RxnRate_EQ195',
 'RxnRate_EQ205',
 'RxnRate_EQ206',
 'RxnRate_EQ207',
 'RxnRate_EQ364',
 'RxnRate_EQ368',
 'RxnRate_EQ370',
 'RxnRate_EQ381',
 'RxnRate_

In [6]:
for line in diag:
    print(line)

'RxnRate_EQ006                           ', 'GIGCchem',
'RxnRate_EQ007                           ', 'GIGCchem',
'RxnRate_EQ008                           ', 'GIGCchem',
'RxnRate_EQ011                           ', 'GIGCchem',
'RxnRate_EQ017                           ', 'GIGCchem',
'RxnRate_EQ018                           ', 'GIGCchem',
'RxnRate_EQ019                           ', 'GIGCchem',
'RxnRate_EQ075                           ', 'GIGCchem',
'RxnRate_EQ078                           ', 'GIGCchem',
'RxnRate_EQ079                           ', 'GIGCchem',
'RxnRate_EQ083                           ', 'GIGCchem',
'RxnRate_EQ084                           ', 'GIGCchem',
'RxnRate_EQ085                           ', 'GIGCchem',
'RxnRate_EQ101                           ', 'GIGCchem',
'RxnRate_EQ102                           ', 'GIGCchem',
'RxnRate_EQ103                           ', 'GIGCchem',
'RxnRate_EQ104                           ', 'GIGCchem',
'RxnRate_EQ105                           ', 'GIG

In [7]:
file = open("../data_output/Tropchem_ROx_self_rxn.txt","r")
lines = file.readlines()
radical_change_list = []
for i in range(len(lines)):
    line = lines[i]
    _, num = line.split('-->')
    num = num.split('\n')[0]
    radical_change_list.append(num)
print(radical_change_list)

[' -2.000', ' -2.000', ' -2.000', ' -2.000', ' -2.000', ' -2.000', ' 0.000', ' -2.000', ' -2.000', ' -1.500', ' -2.000', ' -0.800', ' -1.200', ' -1.000', ' -1.000', ' -2.000', ' -2.000', ' -1.850', ' -1.850', ' -1.874', ' -1.218', ' -2.000', ' -2.000', ' -2.000', ' -2.000', ' -1.000', ' -1.000', ' -0.990', ' -1.350', ' -1.700', ' -1.500', ' -0.900', ' -1.000', ' -1.000', ' -1.360', ' -0.300', ' -1.000', ' -1.100', ' -1.500', ' 0.000', ' 0.000', ' -2.000', ' -2.000', ' -2.000', ' -2.000', ' -2.000', ' -2.000', ' -0.050', ' -0.600', ' 0.000', ' 0.000', ' -0.200', ' -1.000', ' -1.060', ' -1.700', ' -1.000', ' -2.000', ' -2.000', ' -2.000', ' -2.000', ' -1.000', ' -2.000', ' -1.000', ' -2.000', ' -2.000']


In [8]:
len(ROx_eq_num), len(radical_change_list)

(65, 65)

### Printing of equation in gckpp_Monitor.F90 is not complete but the ranking of rxns is the same in gckpp_Monitor.F90 and Tropchem.eqn.

In [9]:
## Final rxn index and radical change
ROx_EQidx_RadicalChange = pd.DataFrame({'ROx_EQ_idx': ROx_eq_num, 'radical_change': radical_change_list})
ROx_EQidx_RadicalChange

,ROx_EQ_idx,radical_change
0,RxnRate_EQ006,-2.000
1,RxnRate_EQ007,-2.000
2,RxnRate_EQ008,-2.000
3,RxnRate_EQ011,-2.000
4,RxnRate_EQ017,-2.000
...,...,...
60,RxnRate_EQ443,-1.000
61,RxnRate_EQ454,-2.000
62,RxnRate_EQ455,-1.000
63,RxnRate_EQ484,-2.000


In [10]:
ROx_EQidx_RadicalChange.to_csv('../data_output/ROx_EQidx_RadicalChange.csv', index=False)